In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata


## Herunterladen der Dateien: 

In [2]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    treffer = xml.find_all("numberOfRecords")[0].text
    treffer = int(treffer)
    loops = int(treffer/100)
        
    if len(records) < 100:
        
        return records
    
    else:
        
        num_results = 100
        i = 101
        #pbar = tqdm(total=loops)
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            #pbar.update()
            
        return records

In [3]:
myquery = dnb_sru('tit=Pandemie and location=onlinefree and cod=pdf')
print(len(myquery), "Ergebnisse")

773 Ergebnisse


## Speichern des Datendumps: 

In [4]:
# simple dump as string
with open('titel_pandemie_oa_pdf.xml', 'w', encoding="utf-8") as f:
    f.write(str(myquery))